In [1]:
import csv
import numpy as np
import pandas as pd
import re
from gensim.models import KeyedVectors
import morfeusz2
from Levenshtein import distance
from collections import Counter
from sklearn.cluster import DBSCAN
from joblib import Parallel, delayed
import multiprocessing
from tqdm import tqdm  
import os.path
import json
df_org = pd.read_excel('shumee_mckinsey -Aktualizacja 01.03.xlsx', index_col=None, engine='openpyxl')

In [2]:
def finder(phrase):
    return (lambda product_name : str(product_name).find(phrase))
df = df_org[df_org['ID zamówienia'].notna()] # usunięcie wpisów gdzie ID zamówienia == NaN
df = df[df['Nazwa produktu'] != 'Przesyłka pobraniowa']
df = df[df['Nazwa produktu'].map(finder("Kod rabatowy")) == -1] # usuniecie kodow rabatowych
df = df[df['Miasto'] != 'test']
df = df[df['Miasto'] != 'Test']
df = df[df['Miasto'] != 'TEST']
df = df[df['Miasto'] != 'ssss']
df = df[df['Miasto'] != 'asd']
df = df[df['Miasto'] != 'asdasd']
df = df[df['Miasto'] != 'dsss']
df = df[df['Nazwa produktu'].map(finder("TESTOWY PRODUKT")) == -1]
df = df[df['Nazwa produktu'].map(finder("PRODUKT TESTOWY")) == -1]
df = df[df['Cena'] >= 0] # usuniecie kodow rabatowych
# akceptuję produktu z ceną 0 zł, ponieważ są to produkty zamówione osobiście lub też telefonicznie

In [3]:
#pd.set_option("display.max_rows", 1000, "display.max_columns", None)
#df.loc[df['Cena'] < 0.1].head(100)
dfPL = df[df['Kraj'] == 'PL']
productsOrg = dfPL['Nazwa produktu'].unique()
products = {}
for i in range(len(productsOrg)):
    products[productsOrg[i]] = [i]

In [4]:
regex = re.compile('[^a-zA-Złóżźśęąćń]')
remove_list = list(map(lambda x:regex.sub('', x.lower()), ['shumee', 'wallart', 'szt', 'szt', 'PEPCO', 'UMBRA', 'KINGHOFF', 'TXM', 'INVICTA',
       'FMD', 'CZYSTA', 'CZYSTE',
       'MODESTO', 'DO', 'HI', 'WOW!', 'KETER',
       'PALMIYE', 'TAI', 'NEW', 'FOCUS', 'NILOE', 'FERM', 'KASSEL',
       'PLEXO55', 'HELMO', 'SISTENA', 'VALENA',
       'EMAS', 'KLAUSBERG',
       'ACTONA', 'CZYSTY', 'SIEMENS', 'PODWÓJNY', 'STRATOS',
       '\ufeffPEPCO',
       'MOSAIC',
       'TROL', 'CELIANE', 'MOVIT', 'ABIZOL',
       'VIGA', 'KARRE', 'STR',
       'SLM', 'EVA', 'EVA&LOLA',
       'SEDNA', 'DUŻA', 'KPL',
       'NOWOCZESNY', 'PERVOI,',
       'OKRĄGŁY', 'PROSTOKĄTNE', 
       'FUNKCJONALNE', 'VOLTRONIC', 'VOLANTE', 'NOWOCZESNA',
       'UNIWERSALNY', 'NATO', 'TRIXIE', 'DIVERO',      
       'L.O.L.', 'BTO-22', 'SMOBY', 'SENTIA',
       '"EVA&LOLA', "CHILDREN'S", 'STEP2', 'INDUSTRIAL',
       'CARLA', 'ELETTROCANALI', 'AKCENT', 'PURO',
       'AK', 'BAS_', 'ITP', 'GLOBOBAR', 'XQ',
       'DSG', 'BERG', 'PRZYPALENIA', 'DRITTO',
       'TON', 'CZARNA',
       'DURO', 'WASSERKESSEL', 'WINDFORCE',
       'SPAMEL', 'EATON', 'KOMPLET',
       'KPL.', 'UMBRA-',
       'PROWERK', 'SENA', 'FLEXI',
       'SKIN79', 'SANN', 'BIG',
       'MINI',
       'FEBER', 'ACTIVE', 'EXTRA', 'PROFESIONALNA',
       'SCHNEIDER', 'ELMERS', 'LINGLONG', 'DICKIE',
       'FEDERAL', 'NUTRA',
       'FITMIN', 'CERTECH', 'SABUNOL', 'NESTOR', 'VITAKRAFT',
       'PROMOCJA', 'ECO',
       'OUTLET', 'HASBRO',
       '4DOGS', 'BASKERVILLE', 'BAHCO', 'EVERLAST', 'KS',
       'YATO', 'BURG-WÄCHTER', 'LONSDALE', 'SCHÜTTE', 'BRILLIANT',
       'VOREL', 'WANDA', 'RIDDER', 'OVERZOO', 'POWER', 'BMW', 'ZOLUX',
       'FALK', 'WIO-MAR', 'FP', 'BEAPHAR', 'HEX', 'S3092',
       'BE1862', 'AS002', 'L8015',
       'TOP', 'ADBI', 'BRIT', 'WADER', 'CHIPSI',
       'APPLAWS', 'PET',
       'BSX', 'ON', 'OTEO', 'SONATA', 'XXL',
       'S86047', 'FRESH', 'VITAPOL', 'FISHER',
       'DR', 'KARO',
       'ULTIMAX', 'PIATNIK', 'CRSM', 'TOITOYS', 'IMPRESJA',
       'KWAD', 'NATURAL', 'HAM-STAKE', 'CRM',
       'NF4575', 'OASY', 'KAPPA',
       'ASFORA', 'AS', 'SIRIUS',
       'FALA', 'BABY', 'FORIX', 'BOB', 'P-51D', 'F&F', 'FONDUE',
       'HL776', 'UNIWERSALNA', 'SIMBA',
       'VACO', 'LOVATO', 'AQUANT', 'KOS66',
       'ECOTOYS', 'DUPLO',
       'HAPPY',
       'TEKNO', 'MADERA',
       'WALKIE', 'RODZINA',
       'PICTUREKA', 'YKA',
       'SA-1993', 'TY', 'DC',
       'SOKY', 'TERA', 'MICCO', 'SAVIO',
       'TG-3468', 'B360N', 'UW', 'NIEBEZPIECZEŃSTWO',
       'KX-TGC212', 'SX1888', 'NIKE',
       'METEOR', 'BD-R', 'FOTE', 'WAMOD', 'GP', '2D',
       'SA-209671', 'SES', 'FLAMINGO', 'ZMYSŁOWY',
       'SPOKEY', 'TL11149004', 'SELECT', 'MZ', 'HIP', 'VMF',
       'FAVOUR', 'GLOBO', 'M62998_1', 'FLAMINGO,', 'PUS',
       'D&D', 'YAKIMA', 'DUTCH', 'FC', 'ESPADRYLE',
       'GINO', 'ELEGANCKI', 'MONDO',
       'HOBBY', 'MDM', 'MINECRAFT', 'Z',
       'RAVEN.', 'ITALERI', 'FIFA', 'PIKMI', 'I',
       'CLEMENTONI', 'MEHANO', 'U.S.',
       'ADDO', 'REVELL', 'DWUSTOPNIOWA', 'C.BLANCHE',
       'PSI', 'DINOTRUX', 'VALLEJO', 'velda', 'xcm', 'versace'
       'GEL-A-PEEL', 'WYPRZEDAŻ', 'CR7', 'V', 'brder', 'mannesmann', 'gratis', 'little', 'tikes', 'nature', 'pro', 'evo', 'rcr', 'proplus']))
             
mapper = {"LEGO" : "klocki", 'DREWEX-POŚCIEL' : 'pościel', "DDR3" : 'pamięć ram'}

to_split = [' ', ',', '.', '-', '_', '!', '+', '*', '&']
polish_chars_mapper = {'ę':'e', 'ó':'o', 'ł':'l', 'ś':'s', 'ą':'a', 'ż':'z', 'ź':'z', 'ć':'c', 'ń':'n'}

def remove_polish_chars(word):
    return "".join(list(map(lambda x: polish_chars_mapper[x] if x in polish_chars_mapper else x, word)))

def prepare_name_products (product):
    product = product.lower()
    for i in to_split:
        product = product.replace(i, ' ')
    product = list(map(lambda x: regex.sub('', x), product.split(' ')))
    #print (product)
    product = [i for i in product if i not in remove_list]
    product = list(map(lambda x: mapper[x] if x in mapper else x, product))
    #print(product)
    result = []
    for i in product:
        if len(i) > 2:
            result.append(i)
    return " ".join(result)

def map_and_unique(function, collection):
    unique_collection = {}
    for k in collection:
        new_k = function(k)
        if new_k not in unique_collection:
            unique_collection[new_k] = []
        unique_collection[new_k] += collection[k]
    return unique_collection

unique_products = map_and_unique(prepare_name_products, products)
print (len(unique_products))

29297


In [5]:
morfeusz = morfeusz2.Morfeusz()

def get_base_name(name):
    analysis = morfeusz.analyse(name)
    pos = -1
    results = []
    for i, j, (token, base, *_) in analysis:

        # pomijam wszystkie interpretacje słowa poza pierwszym
        if pos == i:
            continue
        else:
            results.append(base.split(':')[0])  # morfeusz kończy formę bazową słowa znakiem ":"
            pos = i

    return ' '.join(results)

In [6]:
# tworzę listę nazw ich odpowiedników z formami podstawowymi
tokenized_products = map_and_unique(get_base_name, unique_products)
tokenized_products = map_and_unique((lambda x: x.lower()), tokenized_products)
print (len(tokenized_products))
tokenized_products

28833


{'ramka clip srebrny': [0, 408, 10091],
 'fontanna stalić nierdzewny basen ogrodowy': [1, 9437],
 'krzesło biurowy pomarańczowy sztuczny': [2],
 'parawan panelowy biały': [3,
  130,
  786,
  932,
  1570,
  2340,
  3962,
  4730,
  10898,
  10985,
  11256],
 'półka ścienny biały dąb sonoma płyta wiórowy': [4],
 'regał książka biały płyta wiórowy': [5, 9275, 16024, 21366],
 'wieszak ubranie subway multi szary': [6],
 'przypinka zapiąć akryl sztuk': [7],
 'biurko kolor biały naturalny': [8],
 'dywanik sztuczny futro królik ciemnoszary': [9, 17758],
 'chwytak kroić cebula': [10],
 'stojak but teleskopowy drążek aluminiowy': [11],
 'karty czarny piotruś mix super man': [12],
 'puzzle element bingo': [13, 3497, 10102],
 'puzzle element myszka mickey': [14, 9425, 10099],
 'puzzle element patrol': [15, 3498, 9426, 10100],
 'puzzle element świnka pepa': [16, 4706],
 'kosz gabionowy stalić kształt litera': [17, 18858],
 'ściana gabionowy pokrywa galwanizować stal': [18,
  1107,
  4359,
  24387,
 

In [7]:
colors = list(map(remove_polish_chars, ['black', 'color', 'antracytow', 'bialo', 'jasnoszary', 'silver', 'melanz', 'white', 'ciemno', 'ciemny', 'fioletowo', 'niebiesko', 'jasny', 'roz', 'szaro', 'biel', 'mat', 'braz', 'ciemnobezowy', 'grafitowyowy', 'bezow', 'cza', 'czarno', 'pomarancza', 'zielone', 'szarosc', 'czerwona', 'ciemnoszary', 'akwamaryna', 'alabastrowy', 'amarantowy', 'ametystowy', 'antracytowy', 'atramentowy', 'bahama yellow', 'bananowy', 'beżowy', 'biały', 'biskupi', 'błękit', 'bordowy', 'bordo', 'brązowy', 'brunatny', 'brzoskwiniowy', 'buraczkowy', 'burgund', 'bursztynowy', 'bury', 'ceglasty', 'chabrowy', 'chamois', 'cielisty', 'cukierkowy', 'cyjan', 'cyklamen', 'cynamonowy', 'cynobrowy', 'cytrynowy', 'czarny', 'czerwień', 'czerwony', 'eozyna', 'feldgrau', 'fioletowy', 'fiołkowy', 'fuksja', 'grafitowy', 'granatowy', 'grynszpan', 'hebanowy', 'herbaciany', 'indygo', 'jagodowy', 'jasnożółty', 'jaśminowy', 'kanarkowy', 'kardynalski', 'karmazynowy', 'karminowy', 'kasztanowy', 'kawowy', 'khaki', 'kobaltowy', 'koniakowy', 'koralowy', 'krwisty', 'lawendowy', 'lazurowy', 'lila', 'liliowy', 'limonkowy', 'łososiowy', 'magenta', 'mahoń', 'mahoniowy', 'majtkowy', 'makowy', 'malachitowy', 'malinowy', 'marchewkowy', 'marengo', 'miedziany', 'miętowy', 'minia', 'miodowy', 'mleczny', 'modry', 'morelowy', 'morski', 'mosiądzowy', 'mysi', 'niebieski', 'niebieskofioletowy', 'oberżynowy', 'ochra', 'oliwkowy', 'oranż', 'orzechowy', 'palisander', 'patynowy', 'pąsowy', 'perłowy', 'piaskowy', 'pistacjowy', 'platynowy', 'płowy', 'pomarańczowy', 'popielaty', 'porcelanowy', 'poziomkowy', 'purpurowy', 'rdzawy', 'różowy', 'rubinowy', 'rudy', 'sadza angielska', 'seledynowy', 'sepia', 'siarkowy', 'siny', 'siwy', 'słomkowy', 'smolisty', 'spiżowy', 'srebrny', 'stalowy', 'szafirowy', 'szafranowy', 'szary', 'szkarłatny', 'szmaragdowy', 'śliwkowy', 'świniowy', 'tabaczkowy', 'tango', 'truskawkowy', 'turkusowy', 'tycjan', 'ugier', 'ultramaryna', 'umbra', 'winny', 'wiśniowy', 'woskowy', 'wrzosowy', 'zieleń', 'zielony', 'złocisty', 'złoty', 'żółty']))
materials = list(map(remove_polish_chars, ['drewniany', 'tekowy', 'akacja', 'eukaliptusowy', 'zelowy', 'sosnowe', 'aluminiu', 'antracyt', 'eukaliptus', 'debowy', 'topolowy', 'sosna', 'orzech', 'mdf', 'metalowy', 'bukowy', 'PVC', 'aluminium', 'mango', 'plastikowy', 'metal', 'zelazo', 'bambusowy', 'poliweglan', 'stones', 'galwanizowac', 'stalic', 'aluminiowy', 'dab', 'akacjowy', 'papierowy', 'papierowy', 'stal', 'betonowy', 'nierdzewny', 'sztuczny']))
others = ['vintage', 'wzor', 'loreal', 'experta', 'kolor', 'kolorowy', 'profi', 'audi', 'jeep' 'wrangler', 'willys', 'terenowy',
          'land', 'rover', 'defender', 'chicago', 'sta', 'rozm', 'naturalny', 'gxp', 'lity', 'eko', 'eco', 'oxford', 'sztuk',
          'xxcm', 'made', 'italia', 'german', 'soviet', 'wysoki', 'niski', 'lekki', 'luksusowy', 'mala', 'maly']
def color_material_other_remover (product):
    product = product.split(' ')
    result = []
    for i in product:
        if not i in colors and not i in materials and not i in others:
            result.append(i)
    return " ".join(result)

def words3(product):
    return " ".join(product.split(' ')[:3])

In [8]:
tokenized_productsWPC = map_and_unique(remove_polish_chars, tokenized_products)
tokenized_productsWC = map_and_unique(color_material_other_remover, tokenized_productsWPC)
tokenized_productsWC = map_and_unique(words3, tokenized_productsWC)
print (len(tokenized_productsWC))
tokenized_productsWC

16324


{'ramka clip': [0, 408, 10091, 4994, 11465],
 'fontanna basen ogrodowy': [1, 9437],
 'krzeslo biurowy': [2],
 'parawan panelowy': [3,
  130,
  786,
  932,
  1570,
  2340,
  3962,
  4730,
  10898,
  10985,
  11256,
  129,
  2130,
  2587,
  3323,
  4993,
  7151,
  11145,
  12521,
  12848,
  23896,
  374,
  1671,
  3218,
  3574,
  3655,
  3689,
  11489,
  17767,
  22775,
  26386,
  29130,
  383,
  1110,
  1489,
  1563,
  1595,
  2480,
  3680,
  4722,
  4759,
  8310,
  11483,
  11564,
  12151,
  16139,
  18605,
  29546,
  1409,
  2382,
  4635,
  15012,
  7938,
  18147,
  27967],
 'polka scienny sonoma': [4, 1598, 3210, 9282, 15102, 17899, 25593],
 'regal ksiazka plyta': [5,
  9275,
  16024,
  21366,
  18505,
  24546,
  840,
  6299,
  16905,
  19528,
  23457,
  5645,
  6442,
  9439,
  15762,
  11276,
  15958,
  17795,
  27345,
  4065,
  4241,
  18236,
  18284,
  15309,
  15364,
  16336,
  17113],
 'wieszak ubranie subway': [6],
 'przypinka zapiac akryl': [7, 3503, 3504],
 'biurko': [8,
  77

In [9]:
productsDB = list(tokenized_productsWC.keys())
number_of_products = len(productsDB)
distanceMatrix = None
special_categories = "namiotowy"

def products_distance(prod1, prod2):
    prod1 = list(prod1.split(' '))
    prod2 = list(prod2.split(' '))
    if (len(prod1) > len(prod2)):
        pom = prod1
        prod1 = prod2
        prod2 = pom
    if (len(prod1) == 0):
        return 2000
    summary_distance = 0
    for i in prod1:
        mn_dist = 0
        if len(i) > 0:
            for j in prod2:
                if i == j and i[-1] != 'y':
                    dst = 2
                else: 
                    dst = 1-min(1, distance(i, j)/max(len(i), len(j)))
                if (dst < 0.4):
                    dst = 0.0
                mn_dist = max(dst, mn_dist)
        summary_distance += mn_dist
    result = 2000-(summary_distance/len(prod1)*1000)
    return max(0, result)

def calculate_one_row_distance(i):
    global number_of_products, productsDB
    m = np.full(number_of_products, -1)
    m[i] = 0.0
    for j in range(i, number_of_products):
        m[j]=products_distance(productsDB[i], productsDB[j])
    return m

In [10]:
if os.path.isfile('distanceMatrix.npy'):
    print("Reading from file")
    distanceMatrix = np.load('distanceMatrix.npy')
else:
    print("Calculating distance matrix")
    num_cores = multiprocessing.cpu_count()
    distanceMatrix = np.array(Parallel(n_jobs=num_cores)(delayed(calculate_one_row_distance)(i) for i in tqdm(range(number_of_products))))
    i_lower = np.tril_indices(number_of_products, -1)
    distanceMatrix[i_lower] = distanceMatrix.T[i_lower]
    np.save('distanceMatrix.npy', distanceMatrix)
print(distanceMatrix.shape)
distanceMatrix

  0%|          | 0/16324 [00:00<?, ?it/s]

Calculating distance matrix


100%|██████████| 16324/16324 [02:15<00:00, 120.30it/s] 


(16324, 16324)


array([[   0, 2000, 2000, ..., 2000, 2000, 1800],
       [2000,  333, 2000, ..., 2000, 2000, 2000],
       [2000, 2000,  500, ..., 2000, 2000, 2000],
       ...,
       [2000, 2000, 2000, ...,    0, 2000, 1866],
       [2000, 2000, 2000, ..., 2000,    0, 2000],
       [1800, 2000, 2000, ..., 1866, 2000,    0]])

In [11]:
dbscan = DBSCAN(eps=500, min_samples=2, metric='precomputed', n_jobs=1)
clustering = dbscan.fit(distanceMatrix)


In [12]:
mergedDBScanResults = {}
merged2pass = {}

In [13]:
labels = list(clustering.labels_)
output = sorted([(i, j, []) for i, j in zip(labels, productsDB)])
for i in range(len(output)):
    tpl = list(output[i])
    tpl[2] = tokenized_productsWC[tpl[1]]
    output[i] = tpl
counter = 0
diffkeys = set()
for i, j, k in output:
    if i >= 1:
        print (i, j)
        if i not in mergedDBScanResults:
            mergedDBScanResults[i] = []
        mergedDBScanResults[i] += k
        counter += (len(k))
    else:
        key = " ".join(j.split(' ')[:2])
#         key = j.split(' ')[0]
        diffkeys.add(key)
#         print(key)
        if key not in merged2pass:
            merged2pass[key]=[]
        merged2pass[key] += k
print(counter)
print(mergedDBScanResults)

1 fontanna basen
1 fontanna basen leda
1 fontanna basen ogrodowy
1 fontanna leda basen
1 fontanna ogrodowy
1 fontanna ogrodowy forma
1 fontanna ogrodowy ksztalt
1 fontanna ogrodowy kula
1 fontanna ogrodowy leda
1 fontanna ogrodowy nie
1 fontanna ogrodowy nier
1 fontanna ogrodowy pompka
1 fontanna wodospad basen
1 ogrodowy fontanna kulisty
1 prostokatny fontanna basen
1 wodospad fontanna basen
2 bucik przypinka dla
2 maszynka przypinka forma
2 maszynka przypinka okragly
2 maszynka przypinka przypinka
2 przypinka
2 przypinka zapiac akryl
2 zestawic przypinka
3 puzzle element ksieznichki
3 puzzle element ksiezniczka
3 puzzle element plac
3 puzzle element sniezka
3 puzzle element spiaca
3 puzzle element spider
3 puzzle element swinka
3 puzzle element zaplatac
4 sciana gabionowy
4 sciana gabionowy pokrywa
5 dwustronny zegar scienny
5 kinowy zegar scienny
5 scienny zegar ogrodowy
5 zegar scienny
5 zegar scienny blink
5 zegar scienny boswil
5 zegar scienny brazowo
5 zegar scienny design
5 zeg

39 szklany sloik kranik
39 szklany sloik pokrywka
39 szklany sloik zamknac
39 uchwyt kuchnia sloik
40 dywanik tekstylny
40 dywanik tekstylny bez
40 dywanik tekstylny bezowo
40 dywanik tekstylny brazowo
40 dywanik tekstylny capuccino
40 dywanik tekstylny capucino
40 dywanik tekstylny jasno
40 dywanik tekstylny nakrapiany
40 dywanik tekstylny on
40 dywanik tekstylny peczkowac
40 dywanik tekstylny wzore
40 dywanik tekstylny zloto
40 dywanik tekstylny zolto
41 drzwi przesuwny
41 drzwi przesuwny grega
41 drzwi przesuwny sonoma
41 drzwi przesuwny szklo
41 podwojny drzwi przesuwny
41 szklany drzwi przesuwny
41 szopa drzwi przesuwny
42 podstawka pod monitor
42 podstawka pod monitortv
42 podstawkaszafka pod monitortv
42 stolikpodstawka pod monitorrtv
42 stolikpodstawka pod monitortv
43 skladany zbiornik deszczowka
43 stojak zbiornik deszczowka
43 zbiornik deszczowka
43 zbiornik deszczowka stylizowac
43 zbiornik deszczowka wyglad
44 autko
44 autko akrobatyczny frykcja
44 autko akrobatyczny skala

195 szopa ogrodowy mwgoup
195 szopa ogrodowy stala
195 szopa ogrodowy szopka
195 szopa szopka ogrodowy
196 robot akcja
196 robot akcja on
196 zabawka robot akcja
197 sloj nakretka
197 sloj przechowywanie nakretka
198 organizer szufladka
198 organizer szufladka vidaxl
199 klamerka premium
199 klamerka premium on
200 einhell tarcza ciac
200 einhell tarcza pic
200 tarcza ciac
200 tarcza ciac drewno
200 wolfcraft tarcza ciac
201 panel okladzinowy scienny
201 panel scienny
201 panel scienny cullinans
201 panel scienny elipsa
201 panel scienny kwadrat
201 panel scienny maxwell
201 panel scienny oberon
201 panel scienny squares
201 panel scienny valeria
201 panel scienny vaults
201 panel scienny waves
201 scienny panel okladzinowy
202 drapak sizalowy slupek
202 drapak slupek sizalowy
203 tarnik
203 tarnik skosny
203 tarnik wpuscic
204 ukladanka element krowka
204 ukladanka krowka
205 mandolina obieraczka warzywo
205 obieraczka warzywo
206 kola taczki osi
206 kolo taczki osi
207 sztucce el
207

501 biustonosz parisienne rozmiar
501 biustonosz patricia rozmiar
501 biustonosz ramona rozmiar
501 biustonosz rosa rozmiar
501 biustonosz rozmiar
501 biustonosz samanta rozmiar
501 biustonosz sandy rozmiar
501 biustonosz simena rozmiar
501 biustonosz sophie rozmiar
501 biustonosz toscana rozmiar
501 biustonosz tresor rozmiar
501 biustonosz ursula rozmiar
501 biustonosz vanessa rozmiar
501 biustonosz victoria rozmiar
501 biustonosz virginia rozmiar
502 polaroid
502 polaroid pldf
502 polaroid pldfs
502 polaroid pldn
502 polaroid plds
502 polaroid plps
503 intex basenowy lampa
503 intex lampa basenowy
504 flaga wloch
504 flaga wloch maszt
505 sukienka model blackred
505 sukienka model blackwhite
505 sukienka model blue
505 sukienka model cementowy
505 sukienka model ecru
505 sukienka model green
505 sukienka model grey
505 sukienka model light
505 sukienka model limonka
505 sukienka model moe
505 sukienka model numoco
505 sukienka model oberzyna
505 sukienka model sara
505 sukienka model

In [14]:
print("Not clustered:", len(diffkeys))
print("Clustered by DBSCAN:", len(mergedDBScanResults))

Not clustered: 8363
Clustered by DBSCAN: 602


In [15]:
print ("Generated", len(merged2pass)+len(mergedDBScanResults), "categories")

Generated 8965 categories


In [16]:
clustered_productsId = []
for i in mergedDBScanResults:
    clustered_productsId.append(mergedDBScanResults[i])
for i in merged2pass:
    clustered_productsId.append(merged2pass[i])
clustered_products = []
for i in clustered_productsId:
    prods = []
    for j in i:
        prods.append(productsOrg[j])
    print(prods)
    clustered_products.append(prods)

['Fontanna do basenu, stal nierdzewna, 64 x 30 x 52 cm, srebrna', 'Fontanna do basenu, stal nierdzewna, 50 x 30 x 90 cm, srebrna', 'Fontanna do basenu, stal nierdzewna, 64 x 30 x 52', 'Fontanna do basenu, stal nierdzewna, 30 x 9 x 26 c', 'Fontanna do basenu, stal nierdzewna, 50 x 30 x 53', 'Fontanna do basenu, stal nierdzewna, 45 x 30 x 65', 'Fontanna do basenu LED, 15,8x11,8x24 cm', 'Fontanna do basenu LED, 22,5x17x25,5 cm', 'Fontanna do basenu LED, 13,5x13,5x18,5 cm', 'Fontanna do basenu z LED, stal nierdzewna, 30x60x4', 'Fontanna ze stali nierdzewnej do basenu ogrodowego', 'Fontanna ze stali nierdzewnej do basenu ogrodowego, 45x30x60 cm', 'Fontanna LED do basenu PERVOI, biała, 43x31 cm', 'Fontanna LED do basenu od Intex, PP', ' Fontanna ogrodowa, srebrna, 48x34x88 cm, stal nierdzewna', 'Fontanna ogrodowa, srebrna, 48x34x88 cm, stal nierdzewna', 'Fontanna ogrodowa, srebrna, 48x34x123 cm, stal nierdzewna', 'Fontanna ogrodowa, srebrna, 60,2x37x122,1 cm, stal', 'Fontanna ogrodowa w form

['Case Folio na telefon komórkowy Huawei P 20 Pro Cz', 'Case Folio na telefon komórkowy Huawei Y6 2018 Czarny']
['Caseology Apex Case - Etui iPhone 8 / 7 (Ocean Gray)']
['Brązowa casual torba męska na ramię Solier S12']
['ACTIVE CAT olej funkcyjny Omega 3 zestaw 2 sztuk 250 ml Karma uzupełniająca dla kotów - 14.99 zł / sztuka']
['WINDFORCE 235/55ZR17 CATCHFORS AllSeason 103W XL TL #E 3PMSF WI1383H1']
['Caterpillar Wywrotka Take-A-Part 80466']
['Cavalli Class - B0IQB62991124_C251_306 36']
['Cegła do pizzy z gałką do pizzy']
['Cele strzelnicze, 2 cele, okrągły i kaczka', ' Cele strzelnicze, obrotowe, automatyczny reset, 8+1', 'Cele Strzelnicze, Obrotowe, Automatyczny Reset, 8+1', 'Cele strzelnicze, obrotowe, automatyczny reset, 8+']
['Centralny zamek i 2 piloty do VW Skoda Audi', 'Centralny zamek i 2 piloty', 'Centralny zamek 4 silniczki i 2 piloty VW / Audi /']
[' Ceramiczna muszla klozetowa, narożna, biała', 'Ceramiczna muszla klozetowa narożna biała', 'Ceramiczna muszla klozetowa, nar

['Gra Na końcu języka']
['PEPCO Gra krokodyl', 'PEPCO - Gra krokodyl']
['Gra Kwartet Koty 1289000497', 'Gra Kwartet Psy GXP-529784']
['Gra List miłosny - edycja rozszerzona']
['Gra Looping Louie']
['Gra Łowienie Pływających Kup Gadget and Gifts', 'PEPCO Gra łowienie rybek (24 rybki)', 'PEPCO - gra łowienie rybek (24 rybki)', 'PEPCO PEPCO Gra łowienie rybek (24 rybki)', 'PEPCO PEPCO gra łowienie rybek (24 rybki) ', 'Gra Łowienie Rybek Stolik Wodny Gra Świeci 25 Elem']
['GRA MAFIA GR003']
['Gra Memory Literki Małe i Duże']
['Gra O mój zboże!']
['Gra Monopoly FC Barcelona 2018', 'Gra Monopoly Junior Electronic Banking', 'Gra Monopoly Junior Electronic Banking GXP-620989']
['Gra w Paleontologię dla Dzieci']
['Gra Pandemic: Upadek Rzymu']
['Gra Państwa-miasta']
['Gra Paszczaki']
['Gra Planszowa Cluedo Junior Hasbro (ES)', 'Gra Planszowa Connect 4 Hasbro', 'Gra Planszowa Gnaj Robaczku ZOCH Dla Dzieci 4 Graczy', 'Gra Planszowa Master Mind Cayro', 'Gra Planszowa Pamięciowa Skubane Kurczaki ZOC

['30 impregnowanych sztachet, 100 cm, drewno FSC 276']
['Imprezowy Balon z Białą Girlandą LED Th3 Party']
['Incase Hardshell Case - Obudowa MacBook Air 13 Ret']
['Industrialna lampa, żelazo i lite drewno mango, czarna, E27']
['InnovaGoods Nawilżacz Aromaterapeutyczny z Efektem']
['Integral pendrive USB 16 GB Fusion USB 3.0 Read sp']
['Integral Smartphone and Tablet - Karta pamięci 64 GB microSDHC/XC 90MB/s Class 10 UHS-I U1 + adapter']
['Integral UltimaPro - Karta pamięci 8GB microSDHC/X']
['Inteligentne Wtyczki WiFi z Pilotem Smartwares SH5']
['Intensywna Kuracja Regenerująca Haircare Argán Postquam (100 ml)']
['Little Tikes Interaktywne Centrum Sportowe 3w1']
['Interaktywny Duży Pająk RC Zdalnie Sterowany Pilot']
['Interaktywna Gitara Pianinko Gra Świeci Niebieska']
['Interaktywna Mówiąca Lodziarnia 1/6']
['FLAMINGO Interaktywna zabawka do aportowania dla p']
['Intex Akumulatorowa pompka powietrza Quick-Fill, 6']
['Intex Automatyczny odkurzacz do basenu naziemnego,', 'Intex Automatyc

['Kamizelka Sportowa Męska z Efektem Sauny InnovaGoo', 'Kamizelka Sportowa Damska z Efektem Sauny InnovaGo']
['Kanał instalacyjny 25x25 WDK25025RW biały 6191053', 'Kanał instalacyjny 150x60 WDK60150RW biały 6191231', 'Kanał instalacyjny 210x60 WDK60210RW biały 6191266', 'Kanał instalacyjny 230x60 WDK60230RW biały 6191274', 'Kanał instalacyjny 40x25 WDK25040RW biały 6191061', 'Kanał instalacyjny 40x40 WDK40040RW biały 6191126', 'Kanał instalacyjny 60x40 WDK40060RW biały 6191134', 'Kanał instalacyjny 130x60 WDK60130RW biały 6191223', 'Kanał instalacyjny 110x40 WDK40110RW bały 6191150']
['Kanał kablowy DLP 35x80 /bez pokrywy/ biały 010411']
['2-kanałowe piloty do napędu do bramy garażowej, 2']
['Kanapa dwuosobowa, materiałowa, 94 x 67 x 76 cm, b']
['Kanapa rozkładana z dwiema poduszkami, sztuczna skóra, czarna', 'Kanapa rozkładana z dwiema poduszkami, sztuczna skóra, czarna GXP-679879', 'Kanapa rozkładana z dwiema poduszkami, sztuczna sk', 'Kanapa rozkładana z dwiema poduszkami, brązowa, 

...

['Łóżko z materacem, czarne, metalowe, 180 x 200 cm', 'Metalowe łóżko z materacem, 140 x 200 cm, czarne,', 'Łóżko z materacem memory, białe, sztuczna skóra, 180 x 200 cm', 'Metalowe łóżko z materacem memory, 180x200cm, biał', 'Łóżko z materacem memory, brązowe, tkanina, 180 x 200 cm', 'Łóżko z materacem Memory Foam 140x200 cm, ciemnosz', 'Metalowe łóżko z materacem z pianki memory, 140x20', 'Metalowe łóżko z materacem z pianki memory, 180x20', 'Łóżko z materacem z pianką memory, 140x200 cm, sosnowe, białe', 'Łóżko z materacem z pianki memory, 180 x 200 cm, beżowe', 'Łóżko z materacem z pianki memory, 140x200 cm, tkanina, zielone', 'Łóżko z materacem 180 x 200 cm, sztuczna skóra, bi', 'Łóżko z materacem 180 x 200 cm, sztuczna skóra, cz', 'Łóżko z materacem, sosnowe, w stylu meksykańskim,', 'Łóżko z materacem, 140x200 cm, tapicerowane materi', 'Łóżko z materacem, jasnoszare, tkanina, 140 x 200 cm', 'Łóżko z materacem, beżowe, tkanina, 180 x 200 cm', 'Łóżko z materacem VIDA XL, taupe, 90x

['3 Poziomowa Bambusowa Półka', '4 poziomowa bambusowa półka', '3 poziomowa bambusowa półka', '2 poziomowa półka na buty', '5-poziomowa półka, przezroczysta, 40x40x130 cm, szkło hartowane', ' 5-poziomowa półka, przezroczysta, 40x40x130 cm, szkło hartowane', '4-poziomowa półka, przezroczysta, 30x30x100 cm, sz', '5-poziomowa półka, przezroczysta, 40x40x130 cm, sz', '5-poziomowa półka, czarna, 30x30x130 cm, szkło har']
['5-poziomowy regał, 60x37x180 cm, lite drewno mango', '2-poziomowy regał, 80x35x75 cm, lite drewno mango', '3-poziomowy regał na kółkach, 60x35x81 cm, czarny,', '5-poziomowy regał na kółkach, 75x35x155 cm, czarny', '3-poziomowy regał z koszami, biały, 35x35x102 cm, drewniany', '4-poziomowy regał z koszami, ciemnoszary 35x35x125', '2-poziomowy regał z koszami, ciemnoszary 35x35x72', '3-poziomowy regał z koszami, ciemnoszary 35x35x102', '3-poziomowy regał z koszami, biały, 35x35x102 cm,', '4-poziomowy regał z koszami, kwiaty, 35x35x125 cm,', '3-poziomowy regał z koszami, kwi

['Pufa Velvet Alize czarna', 'Pufa Velvet Alize niebieska']
['Puf vidaXL, okrągły, aksamit z haftem, 60x25 cm, rdzawy']
['Puf welurowy 40 cm jasnoszary COROLLA', 'Puf welurowy 47 cm różowy LOVETT', 'Puf welurowy ø 39 cm czarny SOPHIA']
['Puf 30 x 30 x 32 cm ciemnoszary WISCONSIN']
['Puf wyplatany, musztardowy żółty, 50x35 cm, bawełn']
['Puf żółto-beżowy 50 x 35 cm CONRAD']
['Pufa zwierzak biała LAMB', 'Pufa zwierzak szara TRICERATOPS']
['Pure2Improve Mata do ćwiczeń, 182x61 cm, czarna, P2I290000', 'Pure2Improve Mata, NBR, zielone khaki', 'Pure2Improve Mata podłogowa, duża', 'Pure2Improve Mata do puttowania, 500x65 cm, P2I140', 'Pure2Improve Mata do puttowania, 237x80 cm, P2I140']
['Pure2Improve Poduszka do jogi i medytacji, niebieska', 'Pure2Improve Poduszka do jogi i medytacji, niebies']
['Pure2Improve Ręcznik do jogi, antypoślizgowy, różo']
['PEPCO Puszka M - szara']
['Puszka n/t 122x122x44.6 IP67 /4 dławnice/ 6455-11P']
['Puszka n/t hermetycnza pusta z membranami 158x118x']
['INDUST

['Regulowane krzesło biurkowe szkolne z podnóżkiem, białe GXP-678410', 'Regulowane krzesło biurkowe szkolne z podnóżkiem, szare GXP-678410', 'Regulowane krzesło biurowe, kubełkowe z podnóżkiem, złote', 'Regulowane krzesło biurowe, kubełkowe z podnóżkiem', 'Regulowane krzesło biurowe z podnóżkiem, czerwone', 'Regulowane krzesło biurowe z podnóżkiem, zielone', 'Regulowane krzesło biurowe z podnóżkiem czarne', 'Regulowane krzesło biurowe z podnóżkiem, czarne', 'Regulowane krzesło biurowe z podnóżkiem, czarne GXP-678410', 'Regulowane krzesło biurowe z podnóżkiem czarne GXP', 'Regulowane krzesło biurowe z podnóżkiem, szare']
['Regulowany leżak bambusowy', 'Regulowany leżak z poduszką, drewno tekowe, 195x59,5x35 cm', 'Regulowany leżak z poduszką, drewno tekowe, 195x59,5x35 cm GXP-683114']
['Regulowana mata plażowa z oparciem - niebieska']
['Regulowany stojak/wieszak na ubrania, stal nierdzewna, srebrny', 'Regulowany stojak/wieszak na ubrania, stal nierdze']
['Regulowany stół kreślarski i krz

['Zasłony, bordowe, 140x245 cm, 2 szt.', 'Zasłony, bordowe, 140x175 cm, 2 szt.', 'Zasłony, szare, 135x175 cm']
['Zasłona, 140 x 240 cm, bawełna']
['Zasłona na drzwi, bambusowa, 56 x 185 cm', 'Zasłona na drzwi, bambusowa, 100 x 200 cm', 'Zasłona na drzwi, bambusowa, 100 x 220 cm', 'Zasłona na drzwi, bambusowa, 90 x 220 cm', 'Zasłona na drzwi, bambusowa, 120 x 220 cm', 'Zasłona na drzwi, bambusowa, 100x220 cm', 'shumee Zasłona na drzwi, bambusowa, 100 x 220 cm', 'Zasłona Na Drzwi, Bambusowa, 90 X 220 Cm', ' Zasłona na drzwi, bambusowa, 56 x 185 cm', 'Zasłona na drzwi, bambusowa, 56x185 cm', 'Zasłona na drzwi, bambusowa, 100 x 220 cm GXP-6764', 'Zasłona na drzwi, bambusowa, 56 x 185 cm GXP-67643', 'Zasłona na drzwi, bambus, 90 x 200 cm']
['Zasłony z dzianiny osnowowej, 2 szt., 140x175 cm, białe gałązki', 'Zasłony z dzianiny osnowowej, 2 szt., 140x225cm, k', 'Zasłony z dzianiny osnowowej, 2 szt, 140x175cm, kr', 'Zasłony z dzianiny osnowowej, 2 szt., 140x245cm, k']
['Zasłony dziecięce, 2 sz

In [17]:
with open('clustered_products.json', 'w') as outfile:
    json.dump(clustered_products, outfile)

In [ ]:
[] -> [[]]